In [ ]:
import numpy as np
import tensorflow as tf
import os

In [ ]:
file = R'D:\SSD downloads\Processed Chest X-ray 14\Updated Processed Chest X-rays 1.npz'

#Creating training data to be used for hyper parameter tuning

loaded_array = np.load(file)
train_image_data = []
train_label = []
for key in loaded_array:
    if "Label" in key:
        train_label.append(loaded_array[key].tolist())
    else:
        train_image_data.append(loaded_array[key])    
y_train = train_label
y_train = np.array(y_train)
X_train = train_image_data
X_train = tf.convert_to_tensor(X_train, dtype=tf.float32)
y_train = tf.convert_to_tensor(y_train, dtype=tf.int32)

In [ ]:
#Creating testing data to be used for hyper parameter evaluation

file = R'D:\SSD downloads\Processed Chest X-ray 14\Updated Processed Chest X-rays 117.npz'
X_test = []
y_test = []
loaded_array = np.load(file)
for key in loaded_array:
    if "Label" in key:
        y_test.append(loaded_array[key].tolist())
    else:
        X_test.append(loaded_array[key])    
y_test = np.array(y_test)

In [ ]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Define the hyperparameters to tune
learning_rates = [0.001, 0.01, 0.1]
batch_sizes = [16, 32, 64]
num_epochs = [10, 20, 30]

# Initialize variables to store the best hyperparameters and model performance
best_accuracy = 0
best_learning_rate = None
best_batch_size = None
best_epochs = None
best_model = None


y_test = np.array(y_test)
X_test = tf.convert_to_tensor(X_test, dtype=tf.float32)
#y_test = tf.convert_to_tensor(y_test, dtype=tf.float32)


# Hyperparameter tuning loop
for learning_rate in learning_rates:
    for batch_size in batch_sizes:
        for epochs in num_epochs:
            # Create the model
            model = tf.keras.Sequential()
            model.add(tf.keras.layers.Dense(14, activation="softmax", input_shape=(14 * 14 * 2048,)))

            # Compile the model
            model.compile(optimizer='adam', loss="categorical_crossentropy", metrics=['accuracy'])

            # Train the model
            model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=3)

            # Evaluate the model on the test set
            y_pred = model.predict(X_test)
            y_pred = y_pred.astype(int)
            accuracy = accuracy_score(y_test, y_pred)

            # Check if the current model has the best accuracy so far
            if accuracy > best_accuracy:
                best_accuracy = accuracy
                best_learning_rate = learning_rate
                best_batch_size = batch_size
                best_epochs = epochs
                best_model = model

# Print the best hyperparameters and accuracy
print("Best Hyperparameters:")
print("Learning Rate:", best_learning_rate)
print("Batch Size:", best_batch_size)
print("Number of Epochs:", best_epochs)
print("Best Accuracy:", best_accuracy)